In [1]:
from data_loader import *
from keras.models import Model, load_model
from keras import backend as K
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import HoverTool

/home/morads/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [48]:
BOOKEH_COLORS = {
    '-10': 'black', 
    '-6': 'blue', 
    '-2': 'red', 
    '2': 'green', 
    '6,10': 'orange'   
}


BOOKEH_SHAPES = {
       '-10': 1, 
    '-6': 1, 
    '-2': 1, 
    '2': 1, 
    '6,10': 1
}

In [49]:
def tsne_model(model, data, pca_dim=50, tsne_dim=2, preds = False, layer_index = -2):
    """
    Does tsne reduction
    Parameters:
    model (keras model)
    data (np array): input data for the model
    layer_name: name of the output layer to do tsne on. 'None' will use 2nd to last layer, before final dense layer
 
    pca_dim (int): first dimensions are reduced using PCA to this number of dimensions
    tsne_dim (int): final desired dimension after doing tsne reduction
    
    Returns:
    np.array: Shape (data.shape[0], tsne_reduc); sample points in reduced space
    """
    
    layer_name = model.layers[layer_index].name
    
    intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
    
    intermediate_output = intermediate_layer_model.predict(data)

    pca = PCA(n_components= pca_dim, random_state = 214853)    
    output_pca_reduced = pca.fit_transform(intermediate_output)
    
    tsne = TSNE(n_components=tsne_dim, random_state=214853)
    intermediates_tsne = tsne.fit_transform(output_pca_reduced)
    
    if preds:
        return intermediates_tsne, model.predict(data)
    return intermediates_tsne

def load_training_data(data_file,num_samples=None, mods = None):
    testdata = LoadModRecData(data_file, 1., 0., 0., load_snrs=None, num_samples_per_key=num_samples, load_mods = mods)
    train_data = testdata.signalData
    train_labels = testdata.signalLabels[:,0]
    return train_data, train_labels




def open_test_file(test_file, snr_model, filter_snr = True):
    # opens a testfile and if filter_snr is set to true then it will filter 
    # out only samples that have been predicted to have snr 10dB
    f = open(test_file, 'rb')
    testdata = pickle.load(f, encoding='latin1')
    testdata = np.asarray([testdata[i+1] for i in range(len(testdata.keys()))])

    if filter_snr:
        snr_probs = snr_model.predict(testdata)
        snr_preds = np.asarray([np.argmax(snr_prob) for snr_prob in snr_probs])
        testdata = testdata[np.where(snr_preds == 5)]
    
    return testdata

      
def get_mods_test(model, data, mods, classes):
    # takes in test set returns those data samples that classify in the group of mods by model

    preds = model.predict(data)
    preds = np.asarray([np.argmax(pred) for pred in preds])
    labels = np.asarray([classes[pred] for pred in preds])

    idx = []
    for i,labl in enumerate(preds):
        if labl in mods:
            idx.append(i)
    idx = np.asarray(idx)
    
    return data[idx]


def separate_labels(tsne_output, labels, preds):
    # splits tsne output, labels, and preds into subarrays that have the same labels
    unique_labels = np.unique(labels)
    
    tsne_subar = [0] * unique_labels.shape[0]
    label_subar = [0] * unique_labels.shape[0]
    preds_subar = [0] * unique_labels.shape[0]
    indices = [0] * unique_labels.shape[0]
    for i,label in enumerate(unique_labels):
        idx = np.where(labels==label)
        
        tsne_subar[i] = tsne_output[idx]
        label_subar[i] = labels[idx]
        preds_subar[i] = preds[idx]
        indices[i] = idx[0]
        
    return tsne_subar, label_subar, indices, preds_subar

In [50]:
model_path = 'snr0048.h5'
model = load_model(model_path)

In [51]:
train_file_path = "/datax/yzhang/training_data/training_data_chunk_14.pkl"
# train_file_path = "/datax/yzhang/army_challenge/training_data/training_data_chunk_14.pkl"

num_samples_from_train = 100

train_mod_group = 0
val_mod_group = 0
test_mod_group = 2

num_files = 13
# num_files = 5

blc = 0

CLASSES = ['16PSK', '2FSK_5KHz', '2FSK_75KHz', '8PSK', 'AM_DSB', 'AM_SSB', 'APSK16_c34',
 'APSK32_c34', 'BPSK', 'CPFSK_5KHz', 'CPFSK_75KHz', 'FM_NB', 'FM_WB',
 'GFSK_5KHz', 'GFSK_75KHz', 'GMSK', 'MSK', 'NOISE', 'OQPSK', 'PI4QPSK', 'QAM16',
 'QAM32', 'QAM64', 'QPSK']



mods_for_train = 4

mods1 = [i for i in range(mods_for_train)]
mods1.append(17)
mods1 = np.asarray(mods1)

mods2 = np.arange(mods_for_train,24)

mods3 = np.arange(24)

all_mods = [mods1, mods2, mods3]


train_modulations = np.asarray([CLASSES[i] for i in all_mods[train_mod_group]])
val_modulations = np.asarray([CLASSES[i] for i in all_mods[val_mod_group]])
test_modulations = np.asarray([CLASSES[i] for i in all_mods[test_mod_group]])
test_modulations = np.asarray(["NOISE"])

snrDict = {
    -10: 0,
     -6: 1,
     -2: 2,
      2: 3,
      6: 4,
     10: 4
}

snr_values = None
labelDict = ['-10', '-6', '-2',  '2','6,10']

def load_training_data_snr(data_file,labelDict,num_samples=None, mods = None, snrDict=None):
    testdata = LoadModRecData(data_file, 1., 0., 0., load_snrs=None, num_samples_per_key=num_samples, load_mods = mods, snrDict=snrDict)
    train_data = testdata.signalData
    train_labels = testdata.snrLabels
    train_labels = np.argmax(train_labels, axis=1)
    train_labels = np.asarray([labelDict[i] for i in train_labels])
    return train_data, train_labels


train_data, train_labels = load_training_data_snr(train_file_path,labelDict=labelDict,num_samples=num_samples_from_train,mods=test_modulations, snrDict=snrDict)

[Data Loader] - Loading Datafile,  /datax/yzhang/training_data/training_data_chunk_14.pkl
[Data Loader] - Counting Number of Examples in Dataset...
[Data Loader] - Number of Examples in Dataset: 600
[Data Loader] - [Modulation Dataset] Adding Collects for: NOISE
[Data Loader] - Converting to numpy arrays...
[Data Loader] - Shuffling Data...
[Data Loader] - Splitting Data...
[Data Loader] - Train Size: 600 Validation Size: 0 Test Size: 0
[Data Loader] - Done.



In [52]:
# pick layer index
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2, 1024)      0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 2, 1024, 1)   0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 2, 512, 64)   960         reshape_1[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 2, 256, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

In [53]:
#########################
# change with new model #
#########################
layer_index = -3

In [54]:
data = train_data
labels = train_labels

print(data.shape, labels.shape)

(600, 2, 1024) (600,)


In [55]:
tsne_output, preds = tsne_model(model=model,data=data, preds=True, layer_index=layer_index)

# round to 3 decimals
preds = np.around(preds, decimals=3)

# turn each prediction into a list of strings
preds = np.asarray([[labelDict[i] +": " + str(pred[i]) for i in range(len(pred))] for pred in preds])

[[0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]]
[['-10: 0.0' '-6: 0.0' '-2: 1.0' '2: 0.0' '6,10: 0.0']
 ['-10: 0.0' '-6: 0.0' '-2: 0.0' '2: 0.0' '6,10: 1.0']]


In [56]:
tsne_sub, labels_sub, indices, preds_sub = separate_labels(tsne_output, labels, preds)

In [57]:
legend = False # will hide legend if it gets annoying
legend = 'label'


In [58]:
tooltips = [("p", "(@x, @y)"),("label", "@label"),("index", "@index"),("pred", "@pred")]

hover_tsne = HoverTool(tooltips = tooltips) 
tools_tsne = [hover_tsne, 'pan', 'wheel_zoom', 'reset','box_zoom','save']
p = figure(plot_width=700, plot_height=700, tools=tools_tsne)
output_notebook()

for i in range(len(tsne_sub)):
    tsne_ = tsne_sub[i]
    labels_ = labels_sub[i]
    indices_ = indices[i]
    preds_ = preds_sub[i]
    labl = labels_[0]

    source_train = ColumnDataSource(
        data=dict(
            x = tsne_[:,0],
            y = tsne_[:,1],
            index = indices_,
            label = labels_,
            pred = preds_
        )
    )
    

    shape = BOOKEH_SHAPES[labl]
    if shape == 1:
        p.circle('x', 'y', size=7, fill_color=BOOKEH_COLORS[labl], 
                 alpha=0.9, line_width=0, source=source_train, name="test", legend=legend)
    elif shape == 2:
        p.diamond('x', 'y', size=7, fill_color=BOOKEH_COLORS[labl], 
                 alpha=0.9, line_width=0, source=source_train, name="test", legend=legend)
    elif shape == 3:
        p.cross('x', 'y', size=7, fill_color=BOOKEH_COLORS[labl], 
                 alpha=0.9, line_width=0, source=source_train, name="test", legend=legend)

p.legend.click_policy="hide"

show(p)

Loading BokehJS ...